#### Define the objective function

In [ ]:
from exp_hh_model import HH_model_exp


In [ ]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    arge: a tupleo f additional fixed parameters (prestep_V, step_V0, ...)
    '''
    